In [32]:
import pandas as pd

In [33]:
tiobe_df = pd.read_csv('../../data/tiobeindex_complete.csv')
wiki_df = pd.read_csv('../../data/wiki_complete.csv')
gtrend_df = pd.read_csv('../../data/googletrends_complete.csv')

languages = ['Java', 'JavaScript', 'PHP', 'Python', 'SQL', 'C++', 'C']

In [34]:
tiobe_new_df = tiobe_df[['Month','Java', 'JavaScript', 'PHP', 'Python', 'SQL', 'C++', 'C']]
tiobe_new_df

,Month,Java,JavaScript,PHP,Python,SQL,C++,C
0,2001-06,26.49,1.55,1.90,1.25,2.96,14.20,20.24
1,2001-07,25.03,1.72,1.38,1.13,2.77,16.11,20.77
2,2001-08,24.66,1.66,1.55,1.20,2.38,16.12,20.75
3,2001-09,24.82,1.63,1.55,1.17,2.36,15.85,20.77
4,2001-10,25.68,1.51,1.78,1.28,2.24,16.10,19.75
...,...,...,...,...,...,...,...,...
276,2024-06,8.40,3.32,1.22,15.39,1.75,10.03,9.23
277,2024-07,8.59,3.79,1.15,16.12,2.04,10.34,9.48
278,2024-08,9.16,3.91,1.46,18.04,2.21,10.04,9.17
279,2024-09,9.45,3.92,1.46,20.17,1.94,10.75,8.89


In [35]:
wiki_df.rename(columns={'Date': 'Month'}, inplace=True)
wiki_new_df = wiki_df[['Month','Java', 'JavaScript', 'PHP', 'Python', 'SQL', 'C++', 'C']]
wiki_new_df

,Month,Java,JavaScript,PHP,Python,SQL,C++,C
0,2015-07,2977,2446,5034,1129,1128,1969,576
1,2015-08,4984,2838,7669,1545,2062,4130,714
2,2015-09,6450,3290,7466,2196,2463,5996,673
3,2015-10,6561,3987,9579,1985,3439,6303,768
4,2015-11,5726,3831,9131,2187,3512,5071,752
...,...,...,...,...,...,...,...,...
107,2024-06,2110,2508,2662,3691,1238,1438,1000
108,2024-07,1936,2747,2745,3412,1279,1306,1194
109,2024-08,1585,2385,1703,2306,854,1879,1089
110,2024-09,1843,2148,1765,2405,612,2131,1273


In [36]:
gtrend_new_df = gtrend_df[['Month','Java', 'JavaScript', 'PHP', 'Python', 'SQL', 'C++', 'C']]
gtrend_new_df

,Month,Java,JavaScript,PHP,Python,SQL,C++,C
0,2015-07,100,12,40,15,67,37,41
1,2015-08,84,13,43,16,77,44,40
2,2015-09,83,14,45,19,81,71,49
3,2015-10,85,15,46,19,87,100,54
4,2015-11,86,16,51,20,94,88,49
...,...,...,...,...,...,...,...,...
107,2024-06,16,20,18,67,58,31,32
108,2024-07,15,21,19,66,61,29,28
109,2024-08,13,20,17,61,55,44,32
110,2024-09,14,18,16,73,50,63,38


In [37]:
datas = [tiobe_new_df, wiki_new_df, gtrend_new_df]

In [38]:
results = []

for data in datas: 
    columns_to_normalize = data.columns[1:]

    data_normalized = data.copy()
    data_normalized[columns_to_normalize] = (
        data[columns_to_normalize] - data[columns_to_normalize].min()
    ) / (data[columns_to_normalize].max() - data[columns_to_normalize].min())

    results.append(data_normalized)

In [39]:
wiki = results[0]
tiobe = results[1]
gtrend = results[2]

In [40]:
def classify_tiobe(value, low_threshold):
    if value <= low_threshold:
        return 'low'
    else:
        return 'high'

In [41]:
for language in languages:
    wiki_subset = wiki[['Month', language]].rename(columns={language: f'{language}_wiki'})
    tiobe_subset = tiobe[['Month', language]].rename(columns={language: f'{language}_tiobe'})
    gtrend_subset = gtrend[['Month', language]].rename(columns={language: f'{language}_gtrend'})

    merged_df = pd.merge(tiobe_subset, wiki_subset, on='Month')
    merged_df = pd.merge(merged_df, gtrend_subset, on='Month')

    low_threshold = merged_df[f'{language}_tiobe'].quantile(0.5)  

    merged_df[f'{language}_tiobe_label'] = merged_df[f'{language}_tiobe'].apply(
        lambda x: classify_tiobe(x, low_threshold)
    )

    low_threshold = merged_df[f'{language}_wiki'].quantile(0.5)  

    merged_df[f'{language}_wiki_label'] = merged_df[f'{language}_wiki'].apply(
        lambda x: classify_tiobe(x, low_threshold)
    )

    low_threshold = merged_df[f'{language}_gtrend'].quantile(0.5)  

    merged_df[f'{language}_gtrend_label'] = merged_df[f'{language}_gtrend'].apply(
        lambda x: classify_tiobe(x, low_threshold)
    )

    merged_df.to_csv(f"../../data/normalized/{language}.csv", index=False)
